In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

os.environ["SM_FRAMEWORK"] = "tf.keras"

In [2]:
from tools.config import setup

from treeseg.data.dataset import prepare_datasets
from treeseg.data.imagepaths import get_image_label_paths

from treeseg.modeling.builder import resume_or_load
from treeseg.modeling.trainer import trainer
from treeseg.evaluation.evaluator import evaluator

Segmentation Models: using `tf.keras` framework.


In [3]:
def run(conf, eval_only):
    assert os.path.exists(
        conf.data_folder
    ), f"Data folder {conf.data_folder} does not exist."

    if not os.path.exists(conf.output_dir):
            os.makedirs(conf.output_dir)

    train_images, train_labels, test_images, test_labels = get_image_label_paths(
        conf.data_folder
    )

    train_dataset, val_dataset, test_dataset = prepare_datasets(
        train_images, train_labels, test_images, test_labels, conf
    )

    model = resume_or_load(conf)

    print(model)
    
    if eval_only:
        print("Evaluation-only mode started.")

        evaluator(model, test_dataset, len(test_images), conf.test_batch_size, conf.threshold)

    else:
        print("Training mode started.")
        
        trainer(model, train_dataset, val_dataset, len(train_images), conf)


if __name__ == "__main__":
    config_file_path = "../configs/kokonet_bs8_cs256.txt"

    conf = setup(config_file_path)

    # Modified Config Variables for Local Execution
    conf.data_folder = "/Users/anisr/Documents/AerialImageModel_ITD"
    conf.output_dir = os.path.join("..", conf.output_dir)

    eval_only = False

    run(conf, eval_only)

No checkpoint found. Proceeding without loading weights.
<Functional name=functional, built=True>
Training mode started.
Epoch 1/100


ValueError: Expected all entries in the `metrics` list to be metric objects. Received instead:
metrics=[[<segmentation_models.metrics.IOUScore object at 0x302cb9a00>, <segmentation_models.metrics.FScore object at 0x302cf23c0>]]